In [21]:
import numpy, pandas, sys, cv2, os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
import keras

In [29]:
neu_data_dir = 'neu-data'
train_dir = os.path.join(neu_data_dir, 'train')
val_dir = os.path.join(neu_data_dir, 'val')
test_dir = os.path.join(neu_data_dir, 'test')
img_width, img_height = 200, 200
input_shape = (img_width, img_height, 3)
epochs = 50
batch_size = 16
nb_train_samples = 1254
nb_validation_samples = 270    
nb_test_samples = 276
classes = 6

In [30]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [31]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [32]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 1254 images belonging to 6 classes.
Found 270 images belonging to 6 classes.
Found 276 images belonging to 6 classes.


In [33]:
check_point = keras.callbacks.ModelCheckpoint('LeNet3', monitor='acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [34]:
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=nb_validation_samples // batch_size,
        verbose=1, 
        callbacks=[check_point])

Epoch 1/50
78/78 [==============================] - 10s 127ms/step - loss: 1.7005 - acc: 0.2471 - val_loss: 1.7959 - val_acc: 0.1953

Epoch 00001: acc improved from -inf to 0.24637, saving model to LeNet3
Epoch 2/50
78/78 [==============================] - 10s 125ms/step - loss: 1.5276 - acc: 0.3739 - val_loss: 1.8452 - val_acc: 0.3125

Epoch 00002: acc improved from 0.24637 to 0.37157, saving model to LeNet3
Epoch 3/50
78/78 [==============================] - 10s 124ms/step - loss: 1.3698 - acc: 0.4480 - val_loss: 1.7619 - val_acc: 0.4727

Epoch 00003: acc improved from 0.37157 to 0.45153, saving model to LeNet3
Epoch 4/50
78/78 [==============================] - 10s 123ms/step - loss: 1.1156 - acc: 0.6007 - val_loss: 1.7481 - val_acc: 0.6016

Epoch 00004: acc improved from 0.45153 to 0.60420, saving model to LeNet3
Epoch 5/50
78/78 [==============================] - 9s 122ms/step - loss: 1.0189 - acc: 0.6263 - val_loss: 2.9351 - val_acc: 0.5547

Epoch 00005: acc improved from 0.60420

78/78 [==============================] - 9s 112ms/step - loss: 0.4056 - acc: 0.8704 - val_loss: 1.7700 - val_acc: 0.7656

Epoch 00045: acc did not improve
Epoch 46/50
78/78 [==============================] - 9s 113ms/step - loss: 0.4204 - acc: 0.8606 - val_loss: 1.9516 - val_acc: 0.7656

Epoch 00046: acc did not improve
Epoch 47/50
78/78 [==============================] - 9s 112ms/step - loss: 0.4215 - acc: 0.8516 - val_loss: 1.7989 - val_acc: 0.7227

Epoch 00047: acc did not improve
Epoch 48/50
78/78 [==============================] - 9s 113ms/step - loss: 0.3955 - acc: 0.8606 - val_loss: 1.8647 - val_acc: 0.7773

Epoch 00048: acc did not improve
Epoch 49/50
78/78 [==============================] - 9s 114ms/step - loss: 0.3139 - acc: 0.9022 - val_loss: 1.9250 - val_acc: 0.7148

Epoch 00049: acc improved from 0.90065 to 0.90224, saving model to LeNet3
Epoch 50/50
78/78 [==============================] - 9s 112ms/step - loss: 0.3494 - acc: 0.8878 - val_loss: 1.9085 - val_acc: 0.7852

Ep

In [35]:
import h5py


In [36]:
model.save_weights("lenet3.h5")

In [37]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

In [38]:
print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

Точность на тестовых данных: 86.76%
